# Fine-Tune Gemma using Hugging Face Transformers and QloRA

### Imports

In [1]:
# Install Pytorch & other libraries
%pip install "torch>=2.4.0" tensorboard

# Install Gemma release branch from Hugging Face
%pip install "transformers>=4.51.3"

# Install Hugging Face libraries
%pip install  --upgrade \
  "datasets>=3.3.2" \
  "accelerate==1.4.0" \
  "evaluate==0.4.3" \
  "bitsandbytes==0.45.3" \
  "trl==0.21.0" \
  "peft==0.14.0" \
  "protobuf==5.29.1" \
  "fsspec==2025.3.0" \
  python-Levenshtein \
  sentencepiece

In [2]:
import os
os.environ["HF_HUB_DISABLE_XET"] = "1"

import jax
print("JAX backend:", jax.default_backend())
print("JAX devices:", jax.devices())

JAX backend: gpu
JAX devices: [CudaDevice(id=0)]


In [ ]:
from google.colab import drive
from google.colab import userdata
from huggingface_hub import login

import os
from pathlib import Path
import json
import random
import re
import sqlite3

import torch
from datasets import load_dataset
import pandas as pd
import Levenshtein

from huggingface_hub import login
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    AutoModelForImageTextToText,
    BitsAndBytesConfig,
)
from peft import LoraConfig
from trl import SFTConfig, SFTTrainer

# Login into Hugging Face Hub
hf_token = userdata.get('HF_TOKEN') # If you are running inside a Google Colab
login(hf_token)

### Setup Repo Structure

In [4]:
drive.mount('/content/drive')

ROOT_DRIVE_DIR = "/content/drive/MyDrive/gemma_lora_ft"

TRAIN_DATA_DIR = Path("./data/train")
TEST_DATA_DIR = Path("./data/test")

WIKISQL_DIR = Path("/content/WikiSQL")
DATA_DIR = Path("/content/data")


Mounted at /content/drive


### System/User Prompt with Prompt Builder

In [5]:
# User prompt template (used in both training & inference)

def generate_raw_prompt(question: str, schema_text: str) -> str:
    return f"""
    You are a precise text-to-SQL generator. Using the known schema of the sql database you must output only a valid SQL query and nothing else.

    SQL Database Schema:
    {schema_text}

    User Question:
    {question}

    SQL Query:\n
    """.strip()

### Pretrained Model

In [7]:
# Hugging Face model id
model_id = "google/gemma-3-4b-pt"  # pre-trained (not instruction-tuned)
# For tokenizer we use the instruction-tuned tokenizer
tokenizer_id = "google/gemma-3-4b-it"

# Select model class based on id
if model_id == "google/gemma-3-4b-pt":
    model_class = AutoModelForCausalLM
else:
    model_class = AutoModelForImageTextToText

# Choose dtype based on GPU capability
if torch.cuda.is_available() and torch.cuda.get_device_capability()[0] >= 8:
    torch_dtype = torch.bfloat16
else:
    torch_dtype = torch.float16

# Define model init arguments
model_kwargs = dict(
    attn_implementation="eager", # Use "flash_attention_2" when running on Ampere or newer GPU
    torch_dtype=torch_dtype, # What torch dtype to use, defaults to auto
    device_map="auto", # Let torch decide how to load the model
)

model_kwargs["quantization_config"] = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_compute_dtype=model_kwargs['torch_dtype'],
    bnb_4bit_quant_storage=model_kwargs['torch_dtype'],
)


print("🔄 Loading model...")
model = model_class.from_pretrained(model_id, **model_kwargs)

print("🔄 Loading tokenizer...")
tokenizer = AutoTokenizer.from_pretrained(tokenizer_id)

# Ensure tokenizer has EOS & PAD set correctly for generation & SFT
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "left"


🔄 Loading model...


config.json:   0%|          | 0.00/815 [00:00<?, ?B/s]

`torch_dtype` is deprecated! Use `dtype` instead!


model.safetensors.index.json:   0%|          | 0.00/90.6k [00:00<?, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

...

tokenizer_config.json:   0%|          | 0.00/1.16M [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.69M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/33.4M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/35.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

### Test Pretrained Model with Dummy Prompt

In [8]:
# 1. Build the prompt
test_prompt = "Make a basic SQL Query"

# 2. Tokenize the raw prompt directly
inputs = tokenizer(
    test_prompt,
    return_tensors="pt"
).to(model.device)

# Get the length of the input tokens to slice the output later
input_len = inputs["input_ids"].shape[-1]

# 3. Generate output
with torch.no_grad():
    outputs = model.generate(
        **inputs,
        max_new_tokens=150,
        do_sample=True,    # optional: sampling or greedy
        temperature=0.7,
        pad_token_id=tokenizer.eos_token_id
    )

# 4. Decode and skip the input tokens
# Slice the output tensor to only get the newly generated tokens
generated_tokens = outputs[0][input_len:]
response = tokenizer.decode(generated_tokens, skip_special_tokens=True)

print(response)



[User 0001]

I have a simple question which I have tried to find the answer to, but cannot.

I have a basic query which pulls data from a database table. This query works just fine when I run it from the PHPMyAdmin interface.

However, when I run the same query from my PHP code, it doesn’t return any results.

The database table is set up as follows:

CREATE TABLE <code>users</code> (
<code>id</code> int(11) NOT NULL AUTO_INCREMENT,
<code>username</code> varchar(25) NOT NULL,
<code>password</code> varchar(255) NOT NULL,
<code>firstname</code> varchar(50) NOT NULL,



### Investigate Basic LLM Properties

In [14]:
print(model.config)

Gemma3Config {
  "architectures": [
    "Gemma3ForConditionalGeneration"
  ],
  "boi_token_index": 255999,
  "dtype": "bfloat16",
  "eoi_token_index": 256000,
  "image_token_index": 262144,
  "initializer_range": 0.02,
  "mm_tokens_per_image": 256,
  "model_type": "gemma3",
  "quantization_config": {
    "_load_in_4bit": true,
    "_load_in_8bit": false,
    "bnb_4bit_compute_dtype": "bfloat16",
    "bnb_4bit_quant_storage": "bfloat16",
    "bnb_4bit_quant_type": "nf4",
    "bnb_4bit_use_double_quant": true,
    "llm_int8_enable_fp32_cpu_offload": false,
    "llm_int8_has_fp16_weight": false,
    "llm_int8_skip_modules": null,
    "llm_int8_threshold": 6.0,
    "load_in_4bit": true,
    "load_in_8bit": false,
    "quant_method": "bitsandbytes"
  },
  "text_config": {
    "_sliding_window_pattern": 6,
    "attention_bias": false,
    "attention_dropout": 0.0,
    "attn_logit_softcapping": null,
    "dtype": "bfloat16",
    "final_logit_softcapping": null,
    "head_dim": 256,
    "hidde

In [9]:
# Find the number of trainable and untrainable parameters
trainable_params = 0
non_trainable_params = 0

for param in model.parameters():
    if param.requires_grad:
        trainable_params += param.numel()
    else:
        non_trainable_params += param.numel()

print(f"Total trainable parameters: {trainable_params}")
print(f"Total non-trainable parameters: {non_trainable_params}")

Total trainable parameters: 680094720
Total non-trainable parameters: 905199984


In [10]:
# Find the number of Transformer Layers in the Model
if hasattr(model.config, 'num_hidden_layers'):
    model_depth = model.config.num_hidden_layers
elif hasattr(model.config, 'n_layers'):
    model_depth = model.config.n_layers
else:
    model_depth = "Could not determine model depth from config."

print(f"Number of transformer layers (depth): {model_depth}")

Number of transformer layers (depth): Could not determine model depth from config.


In [11]:
# Find the hidden size of the model
if hasattr(model.config, 'hidden_size'):
    hidden_size = model.config.hidden_size
elif hasattr(model.config, 'd_model'):
    hidden_size = model.config.d_model
else:
    hidden_size = "Could not determine hidden size from config."

print(f"Hidden size: {hidden_size}")

Hidden size: Could not determine hidden size from config.


In [12]:
# Find the number of attention heads within the model
if hasattr(model.config, 'num_attention_heads'):
    num_attention_heads = model.config.num_attention_heads
elif hasattr(model.config, 'num_heads'):
    num_attention_heads = model.config.num_heads
elif hasattr(model.config, 'n_head'):
    num_attention_heads = model.config.n_head
else:
    num_attention_heads = "Could not determine number of attention heads from config."

print(f"Number of attention heads: {num_attention_heads}")

Number of attention heads: Could not determine number of attention heads from config.


In [13]:
# Find the vocab size of the pretrained model
if hasattr(model.config, 'vocab_size'):
    vocab_size = model.config.vocab_size
else:
    vocab_size = "Could not determine vocabulary size from config."

print(f"Vocabulary size: {vocab_size}")

Vocabulary size: Could not determine vocabulary size from config.


### Gemma Model Summary:

The key statistics and network features of the Gemma-3 model are as follows:
*   **Total Trainable Parameters:** 302,124,160
*   **Total Non-Trainable Parameters:** 174,440,448
*   **Number of Transformer Layers (Depth):** 26
*   **Hidden Size:** 1,152
*   **Number of Attention Heads:** 4
*   **Vocabulary Size:** 262,144



### Training Dataset

In [15]:
def normalize_schema(schema_text: str) -> str:
    """
    Convert a synthetic schema description into a WikiSQL-style schema:

        - column_name (TEXT)
        - another_column (TEXT)

    We use a simple heuristic: try to extract tokens in quotes as column names.
    If that fails, fall back to first tokens of lines.
    """
    if schema_text is None:
        return ""

    # Try to extract names in single/double quotes
    columns = re.findall(r"'([^']+)'|\"([^\"]+)\"", schema_text)
    columns = [c[0] or c[1] for c in columns if (c[0] or c[1])]

    # Fallback: use first token of each non-empty line
    if len(columns) == 0:
        lines = [ln.strip() for ln in schema_text.splitlines() if ln.strip()]
        for ln in lines:
            # take the first token, strip punctuation
            tok = ln.split()[0].strip(",:;")
            if tok:
                columns.append(tok)

    # Deduplicate while preserving order
    seen = set()
    unique_cols = []
    for c in columns:
        if c not in seen:
            seen.add(c)
            unique_cols.append(c)

    if not unique_cols:
        return ""

    formatted = "\n".join(f"- {col} (TEXT)" for col in unique_cols)
    return formatted


def normalize_training_sql(sql: str) -> str:
    """
    Normalize synthetic SQL labels to be closer to WikiSQL:
    - Remove trailing semicolons
    - Normalize quotes
    - Force leading SELECT to uppercase
    """
    if sql is None:
        return ""

    sql = sql.strip()
    sql = sql.replace("`", '"')
    sql = sql.replace("[", '"').replace("]", '"')
    sql = sql.rstrip(";")

    if sql.lower().startswith("select"):
        sql = "SELECT" + sql[6:]

    return sql

def build_training_row(sample):
    # 1. Normalize schema and SQL
    schema_fixed = normalize_schema(sample["sql_context"])
    sql_fixed = normalize_training_sql(sample["sql"])

    # 2. Build raw prompt
    prompt = generate_raw_prompt(
        question=sample["sql_prompt"],
        schema_text=schema_fixed,
    )

    # 3. Return a flat table row
    return {
        "prompt": prompt,
        "completion": sql_fixed,
        # TRL expects a single field "text" for SFT:
        "text": prompt + sql_fixed
    }




In [16]:
# Load dataset from the hub
# Load synthetic dataset
print("🔄 Loading synthetic text-to-SQL dataset...")
raw_synth = load_dataset("philschmid/gretel-synthetic-text-to-sql", split="train")

# Shuffle & subselect to a manageable size (consistent with earlier code)
raw_synth = raw_synth.shuffle(seed=42).select(range(50000))

🔄 Loading synthetic text-to-SQL dataset...


README.md:   0%|          | 0.00/737 [00:00<?, ?B/s]

(…)nthetic_text_to_sql_train.snappy.parquet:   0%|          | 0.00/32.4M [00:00<?, ?B/s]

(…)ynthetic_text_to_sql_test.snappy.parquet:   0%|          | 0.00/1.90M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/100000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/5851 [00:00<?, ? examples/s]

In [17]:
print("🔄 Converting synthetic dataset to chat format...")
processed_dataset = raw_synth.map(
    build_training_row,
    remove_columns=raw_synth.column_names
)

# Train / test split on synthetic (for training + internal validation if desired)
# synth_dataset = processed_dataset.train_test_split(test_size=2500 / 12500, seed=42)
# print(synth_dataset)

# Quick sanity check
print("Example training messages:")
print(processed_dataset[0]['prompt'])


🔄 Converting synthetic dataset to chat format...


Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

Example training messages:
You are a precise text-to-SQL generator. Using the known schema of the sql database you must output only a valid SQL query and nothing else.

    SQL Database Schema:
    - Female (TEXT)
- Premium (TEXT)
- Male (TEXT)
- Basic (TEXT)
- Cycling (TEXT)
- 2022-04-01 (TEXT)
- Yoga (TEXT)
- 2022-04-02 (TEXT)
- 2022-04-03 (TEXT)
- 2022-04-04 (TEXT)
- Pilates (TEXT)
- 2022-04-05 (TEXT)
- 2022-04-06 (TEXT)
- 2022-04-07 (TEXT)
- 2022-04-08 (TEXT)
- 2022-04-09 (TEXT)
- 2022-04-10 (TEXT)

    User Question:
    How many members attended each type of class in April 2022?

    SQL Query:


### WikiSQL Dataset

In [18]:
if not os.path.exists(WIKISQL_DIR):
  !git clone https://github.com/salesforce/WikiSQL

if not os.path.exists(DATA_DIR):
  !tar xvjf /content/WikiSQL/data.tar.bz2

Cloning into 'WikiSQL'...
remote: Enumerating objects: 389, done.
remote: Counting objects: 100% (195/195), done.
remote: Compressing objects: 100% (41/41), done.
remote: Total 389 (delta 186), reused 154 (delta 154), pack-reused 194 (from 1)
Receiving objects: 100% (389/389), 50.72 MiB | 15.65 MiB/s, done.
Resolving deltas: 100% (213/213), done.
data/
data/train.jsonl
data/test.tables.jsonl
data/test.db
data/dev.tables.jsonl
data/dev.db
data/test.jsonl
data/train.tables.jsonl
data/train.db
data/dev.jsonl


### SQLite Setup and Table Builder

In [19]:
def load_tables(split="dev"):
    tables_path = DATA_DIR / f"{split}.tables.jsonl"
    tables = {}
    with open(tables_path, "r", encoding="utf-8") as f:
        for line in f:
            obj = json.loads(line)
            tables[obj["id"]] = obj
    return tables


def iter_split(split="dev"):
    q_path = DATA_DIR / f"{split}.jsonl"
    tables = load_tables(split)
    with open(q_path, "r", encoding="utf-8") as f:
        for line in f:
            ex = json.loads(line)
            table = tables[ex["table_id"]]
            yield ex, table


def build_sqlite_from_table(table_obj):
    df = pd.DataFrame(table_obj["rows"], columns=table_obj["header"])
    conn = sqlite3.connect(":memory:")
    df.to_sql("data", conn, index=False, if_exists="replace")
    return conn


AGG_OPS = ["", "MAX", "MIN", "COUNT", "SUM", "AVG"]
COND_OPS = ["=", ">", "<", "OP"]


def logical_to_sql(sql_obj, table_obj, table_name="data"):
    """
    Convert WikiSQL logical form (sel, agg, conds) into a SQL string.
    """
    sel_idx = sql_obj["sel"]
    agg_idx = sql_obj["agg"]
    conds = sql_obj["conds"]

    columns = table_obj["header"]
    sel_col = f'"{columns[sel_idx]}"'
    agg = AGG_OPS[agg_idx]

    select_expr = (agg + " " + sel_col).strip()
    query = f"SELECT {select_expr} FROM {table_name}"

    where_clauses = []
    for col_idx, op_idx, val in conds:
        col_name = f'"{columns[col_idx]}"'
        op = COND_OPS[op_idx]

        if isinstance(val, str):
            v_str = "'" + val.replace("'", "''") + "'"
        else:
            v_str = str(val)

        if op == "OP":
            op = "="

        where_clauses.append(f"{col_name} {op} {v_str}")

    if where_clauses:
        query += " WHERE " + " AND ".join(where_clauses)

    return query


def execute_sql(conn, sql):
    try:
        cur = conn.cursor()
        cur.execute(sql)
        return cur.fetchall(), None
    except Exception as e:
        return None, str(e)


def make_schema_text(table_obj):
    """
    Convert WikiSQL table to the same schema format used for synthetic data,
    so the model sees consistent inputs.
    """
    cols = table_obj["header"]
    return "\n".join(f"- {c} (TEXT)" for c in cols)

In [20]:
ex, tbl = next(iter_split("dev"))
example_gold_sql = logical_to_sql(ex["sql"], tbl)
print("\nWikiSQL sanity check:")
print("Question:", ex["question"])
print("Gold SQL:", example_gold_sql)


WikiSQL sanity check:
Question: What position does the player who played for butler cc (ks) play?
Gold SQL: SELECT "Position" FROM data WHERE "School/Club Team" = 'Butler CC (KS)'


### Evaluation Helper Functions

In [21]:
def clean_sql_output(text: str) -> str:
    """
    Extract a valid SQL statement from generated text.
    Handles:
        - stray whitespace
        - model continuing with commentary
        - model outputting multiple SQL candidates
    """
    text = text.strip()

    # Remove any repeated "SQL Query:" prefix
    text = text.replace("SQL Query:", "").strip()

    # Remove markdown
    text = text.replace("```sql", "").replace("```", "")

    # Only return first SELECT statement
    lines = [ln.strip() for ln in text.splitlines()]

    for ln in lines:
        if ln.upper().startswith("SELECT"):
            return ln

    # Fallback: regex anywhere
    m = re.search(r"SELECT\s+.+", text, flags=re.IGNORECASE)
    return m.group(0).strip() if m else ""


def generate_sql_from_llm(question: str, schema_text: str, model) -> str:
    """
    Generate SQL from a raw text prompt using continuation-style generation.
    """
    # Build the same raw prompt used during training
    prompt = generate_raw_prompt(question, schema_text)

    # Tokenize
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    # Compute input length to slice off the prompt from model output
    input_len = inputs["input_ids"].shape[-1]

    # Generate continuation
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=150,
            do_sample=False,  # deterministic for evaluation
            pad_token_id=tokenizer.eos_token_id
        )

    # Slice off the prompt to isolate model-generated SQL
    gen_tokens = outputs[0][input_len:]
    decoded = tokenizer.decode(gen_tokens, skip_special_tokens=True)

    return clean_sql_output(decoded)


# SQL similarity score evaluator
def tokenize_sql(sql: str):
    sql = sql.lower()
    sql = re.sub(r"[^a-z0-9_*]", " ", sql)
    tokens = sql.split()
    return tokens

def jaccard_similarity(sql1: str, sql2: str):
    t1 = set(tokenize_sql(sql1))
    t2 = set(tokenize_sql(sql2))
    if not t1 and not t2:
        return 1.0
    return len(t1 & t2) / len(t1 | t2)

def levenshtein_similarity(sql1: str, sql2: str):
    if not sql1 and not sql2:
        return 1.0
    dist = Levenshtein.distance(sql1, sql2)
    max_len = max(len(sql1), len(sql2))
    return 1 - (dist / max_len)

def extract_sql_structure(sql: str):
    sql = sql.lower()

    # extract SELECT columns
    select_cols = []
    where_cols = []

    # SELECT fields
    m = re.search(r"select\s+(.*?)\s+from", sql)
    if m:
        select_cols = [
            col.strip().replace('"', '').replace("'", "")
            for col in m.group(1).split(",")
        ]

    # WHERE fields
    m = re.search(r"where\s+(.*)", sql)
    if m:
        conditions = m.group(1).split("and")
        for cond in conditions:
            stripped_cond = cond.strip()
            if stripped_cond: # Only process non-empty conditions
                parts = stripped_cond.split()
                if parts: # Ensure there's at least one word
                    col = parts[0].replace('"', '').replace("'", "")
                    where_cols.append(col)

    # Operators
    ops = []
    for op in ["=", ">", "<", "like"]:
        if op in sql:
            ops.append(op)

    return {
        "select_cols": set(select_cols),
        "where_cols": set(where_cols),
        "ops": set(ops)
    }

def structural_similarity(sql1: str, sql2: str):
    s1 = extract_sql_structure(sql1)
    s2 = extract_sql_structure(sql2)

    # Column match = avg intersection on SELECT & WHERE
    select_match = (
        len(s1["select_cols"] & s2["select_cols"]) /
        max(1, len(s1["select_cols"] | s2["select_cols"]))
    )

    where_match = (
        len(s1["where_cols"] & s2["where_cols"]) /
        max(1, len(s1["where_cols"] | s2["where_cols"]))
    )

    op_match = (
        len(s1["ops"] & s2["ops"]) /
        max(1, len(s1["ops"] | s2["ops"]))
    )

    return {
        "select_match": select_match,
        "where_match": where_match,
        "op_match": op_match,
    }

In [22]:
print("\n🔎 Single WikiSQL example BEFORE training:")
ex, tbl = next(iter_split("dev"))
conn = build_sqlite_from_table(tbl)
schema = make_schema_text(tbl)

gold_sql = logical_to_sql(ex["sql"], tbl)
pred_sql = generate_sql_from_llm(ex["question"], schema, model)

gold_res, ge = execute_sql(conn, gold_sql)
pred_res, pe = execute_sql(conn, pred_sql)

print("Q:", ex["question"])
print("Gold:", gold_sql)
print("Pred:", pred_sql)
print("Gold result:", gold_res)
print("Pred result:", pred_res if pe is None else f"ERROR: {pe}")

The following generation flags are not valid and may be ignored: ['top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.



🔎 Single WikiSQL example BEFORE training:
Q: What position does the player who played for butler cc (ks) play?
Gold: SELECT "Position" FROM data WHERE "School/Club Team" = 'Butler CC (KS)'
Pred: SELECT Position FROM Player WHERE School/Club Team = 'Butler CC (KS)'
Gold result: [('Guard-Forward',)]
Pred result: ERROR: near "Team": syntax error


### Evaluation Suite

In [23]:
def evaluate_execution(model, n=50, split="dev", verbose=False):
    """
    Compute execution accuracy on WikiSQL:
      • sample n random examples from the chosen split
      • generate SQL using the model
      • execute gold SQL and predicted SQL in SQLite
      • count exact result-set matches

    Returns:
        float — execution accuracy in [0, 1]
    """
    examples = list(iter_split(split))
    correct = 0

    for i in range(n):
        # Sample
        ex, tbl = random.choice(examples)
        conn = build_sqlite_from_table(tbl)
        schema = make_schema_text(tbl)

        # Get SQL queries
        gold_sql = logical_to_sql(ex["sql"], tbl)
        pred_sql = generate_sql_from_llm(ex["question"], schema, model)

        # Execute both
        gold_res, ge = execute_sql(conn, gold_sql)
        pred_res, pe = execute_sql(conn, pred_sql)

        # Determine correctness
        is_correct = (ge is None and pe is None and pred_res == gold_res)
        if is_correct:
            correct += 1

        # Logging
        if verbose:
            status = "✅ Correct" if is_correct else "❌ Wrong"
            print(f"[{i+1}/{n}] {status}: {pred_sql}")

    return correct / n



def evaluate_execution_detailed(model, n=50, split="dev"):
    """
    Execute detailed evaluation that includes:
      • execution accuracy
      • lexical similarity (Jaccard & Levenshtein)
      • structural similarity (SELECT column match, WHERE column match)
      • error collection

    Returns:
        {
          "exec_accuracy": float,
          "results": [ { detailed result dict }, ... ]
        }
    """
    examples = list(iter_split(split))
    correct = 0
    results = []

    for i in range(n):
        ex, tbl = random.choice(examples)
        conn = build_sqlite_from_table(tbl)
        schema = make_schema_text(tbl)

        # Gold + Prediction
        gold_sql = logical_to_sql(ex["sql"], tbl)
        pred_sql = generate_sql_from_llm(ex["question"], schema, model)

        # Execution
        gold_res, ge = execute_sql(conn, gold_sql)
        pred_res, pe = execute_sql(conn, pred_sql)
        exec_match = (ge is None and pe is None and gold_res == pred_res)

        if exec_match:
            correct += 1

        # Similarities
        jacc = jaccard_similarity(gold_sql, pred_sql)
        lev = levenshtein_similarity(gold_sql, pred_sql)
        struct = structural_similarity(gold_sql, pred_sql)

        results.append({
            "question": ex["question"],
            "schema": schema,
            "gold_sql": gold_sql,
            "pred_sql": pred_sql,
            "exec_match": exec_match,
            "jaccard": jacc,
            "levenshtein": lev,
            "select_match": struct["select_match"],
            "where_match": struct["where_match"],
            "op_match": struct["op_match"],
            "gold_res": gold_res if ge is None else None,
            "pred_res": pred_res if pe is None else None,
            "gold_error": ge,
            "pred_error": pe,
        })

    return {
        "exec_accuracy": correct / n,
        "results": results
    }



def print_error_examples(results, k=5):
    """
    Pretty-print up to k incorrect examples for debugging.
    """
    errors = [r for r in results if not r["exec_match"]]

    print(f"\nShowing up to {k} error cases (of {len(errors)} total):\n")
    for i, err in enumerate(errors[:k], start=1):
        print("=" * 80)
        print(f"Error #{i}")
        print(f"Question:\n  {err['question']}\n")
        print("Schema:")
        print(err["schema"])
        print("\nGold SQL:", err["gold_sql"])
        print("Pred SQL:", err["pred_sql"])
        print("\nGold result / error:", err["gold_res"], "/", err["gold_error"])
        print("Pred result / error:", err["pred_res"], "/", err["pred_error"])
        print("=" * 80)



def print_similarity_report(results, k=5):
    """
    Show top-k predictions ranked by lexical similarity to gold SQL.
    Useful for detecting partial correctness even when execution fails.
    """
    ranked = sorted(results, key=lambda r: r["jaccard"], reverse=True)

    print(f"\nTop {k} most similar SQL predictions (by Jaccard similarity):\n")
    for i, r in enumerate(ranked[:k], start=1):
        print("=" * 80)
        print(f"Example #{i}")
        print("Question:", r["question"])
        print("Gold:", r["gold_sql"])
        print("Pred:", r["pred_sql"])
        print(f"Jaccard:     {r['jaccard']:.3f}")
        print(f"Levenshtein: {r['levenshtein']:.3f}")
        print(f"SELECT col match: {r['select_match']:.3f}")
        print(f"WHERE col match:  {r['where_match']:.3f}")
        print(f"Operator match:   {r['op_match']:.3f}")
        print("=" * 80)


In [24]:
# Baseline before training
baseline = evaluate_execution_detailed(model, n=10)
print("Baseline execution accuracy:", baseline["exec_accuracy"])
print_error_examples(baseline["results"], k=10)

# Eval on 100 samples
# evaluation = evaluate_execution_detailed(model, n=100)
# print("Execution accuracy:", evaluation["exec_accuracy"])

# print_similarity_report(evaluation["results"], k=5)

Baseline execution accuracy: 0.0

Showing up to 10 error cases (of 10 total):

Error #1
Question:
  When was roshan mahanama born?

Schema:
- Player (TEXT)
- Date of Birth (TEXT)
- Batting Style (TEXT)
- Bowling Style (TEXT)
- First Class Team (TEXT)

Gold SQL: SELECT "Date of Birth" FROM data WHERE "Player" = 'roshan mahanama'
Pred SQL: SELECT Player, Date of Birth FROM Player WHERE Player = 'roshan mahanama'

Gold result / error: [] / None
Pred result / error: None / near "Birth": syntax error
Error #2
Question:
  Name the lowest Market Value (billion $) which has Assets (billion $) larger than 276.81, and a Company of toyota, and Profits (billion $) larger than 17.21?

Schema:
- Rank (TEXT)
- Company (TEXT)
- Headquarters (TEXT)
- Industry (TEXT)
- Sales (billion $) (TEXT)
- Profits (billion $) (TEXT)
- Assets (billion $) (TEXT)
- Market Value (billion $) (TEXT)

Gold SQL: SELECT MIN "Market Value (billion $)" FROM data WHERE "Assets (billion $)" > 276.81 AND "Company" = 'toyota' AN

In [25]:
print_similarity_report(baseline["results"], k=5)


Top 5 most similar SQL predictions (by Jaccard similarity):

Example #1
Question: When was roshan mahanama born?
Gold: SELECT "Date of Birth" FROM data WHERE "Player" = 'roshan mahanama'
Pred: SELECT Player, Date of Birth FROM Player WHERE Player = 'roshan mahanama'
Jaccard:     0.900
Levenshtein: 0.781
SELECT col match: 0.500
WHERE col match:  1.000
Operator match:   1.000
Example #2
Question: what is the school with the location of alexandria?
Gold: SELECT "School" FROM data WHERE "Location" = 'alexandria'
Pred: SELECT School, Location FROM School WHERE Location = 'Alexandria';
Jaccard:     0.857
Levenshtein: 0.682
SELECT col match: 0.500
WHERE col match:  0.333
Operator match:   1.000
Example #3
Question: Which Score has an Opponent of @ houston, and a Record of 2-0?
Gold: SELECT "Score" FROM data WHERE "Opponent" = '@ houston' AND "Record" = '2-0'
Pred: SELECT * FROM Score WHERE Opponent = 'houston' AND Record = '2-0'
Jaccard:     0.833
Levenshtein: 0.763
SELECT col match: 0.000
W

# Model Training

### LoRA Config

In [26]:
peft_config = LoraConfig(
    r=64,
    lora_alpha=32,
    lora_dropout=0.05,
    target_modules=[
        "q_proj",
        "k_proj",
        "v_proj",
        "o_proj",
        "gate_proj",
        "up_proj",
        "down_proj",
    ],
    bias="none",
    task_type="CAUSAL_LM",
    modules_to_save=["lm_head", "embed_tokens"]
)

### SFT Config

In [27]:
from trl import SFTConfig

training_args  = SFTConfig(
    output_dir="gemma-text-to-sql",         # directory to save and repository id
    max_length=512,                         # max sequence length for model and packing of the dataset
    packing=True,                           # Groups multiple samples in the dataset into a single sequence
    num_train_epochs=3,                     # number of training epochs
    per_device_train_batch_size=1,          # batch size per device during training
    gradient_accumulation_steps=4,          # number of steps before performing a backward/update pass
    gradient_checkpointing=True,            # use gradient checkpointing to save memory
    optim="adamw_torch_fused",              # use fused adamw optimizer
    logging_steps=10,                       # log every 10 steps
    save_strategy="epoch",                  # save checkpoint every epoch
    learning_rate=2e-4,                     # learning rate, based on QLoRA paper
    fp16=True if torch_dtype == torch.float16 else False,   # use float16 precision
    bf16=True if torch_dtype == torch.bfloat16 else False,   # use bfloat16 precision
    max_grad_norm=0.3,                      # max gradient norm based on QLoRA paper
    warmup_ratio=0.03,                      # warmup ratio based on QLoRA paper
    lr_scheduler_type="constant",           # use constant learning rate scheduler
    push_to_hub=True,                       # push model to hub
    report_to="tensorboard",                # report metrics to tensorboard
)

### SFT Trainer

In [28]:
trainer = SFTTrainer(
    model=model,
    args=training_args,
    train_dataset=processed_dataset,
    eval_dataset=None,
    processing_class=tokenizer,
    peft_config=peft_config,
)

/usr/local/lib/python3.12/dist-packages/trl/trainer/sft_trainer.py:453: UserWarning: Padding-free training is enabled, but the attention implementation is not set to 'flash_attention_2'. Padding-free training flattens batches into a single sequence, and 'flash_attention_2' is the only known attention mechanism that reliably supports this. Using other implementations may lead to unexpected behavior. To ensure compatibility, set `attn_implementation='flash_attention_2'` in the model configuration, or verify that your attention mechanism can handle flattened sequences.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/trl/trainer/sft_trainer.py:495: UserWarning: You are using packing, but the attention implementation is not set to 'flash_attention_2' or 'kernels-community/vllm-flash-attn3'. Packing flattens batches into a single sequence, and Flash Attention is the only known attention mechanisms that reliably support this. Using other implementations may lead to cross-contaminatio

Adding EOS to train dataset:   0%|          | 0/50000 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/50000 [00:00<?, ? examples/s]

/usr/local/lib/python3.12/dist-packages/trl/trainer/sft_trainer.py:767: UserWarning: Mismatch between tokenized prompt and the start of tokenized prompt+completion. This may be due to unexpected tokenizer behavior, whitespace issues, or special token handling. Verify that the tokenizer is processing text consistently.
  warnings.warn(


Packing train dataset:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [ ]:
# Start training, the model will be automatically saved to the Hub and the output directory
trainer.train()

# Save the final model again to the Hugging Face Hub
trainer.save_model()

# After training, `model` now has LoRA adapters applied in-place.
ft_model = trainer.model

The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'eos_token_id': 1, 'bos_token_id': 2, 'pad_token_id': 0}.
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.


Step,Training Loss
10,0.673700
20,0.622100
30,0.592500
40,0.566800
50,0.590800
60,0.551600
70,0.591500
80,0.567100
90,0.549500
100,0.512900


In [ ]:
os.makedirs(ROOT_DRIVE_DIR, exist_ok=True)

trainer.model.save_pretrained(ROOT_DRIVE_DIR)
tokenizer.save_pretrained(ROOT_DRIVE_DIR)

### Load Trained Model

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel

BASE_MODEL = "google/gemma-3-4b-pt"
ADAPTER_DIR = "gemma-text-to-sql"   # your output dir

print("Loading base model...")
base_model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL,
    device_map="auto",
    torch_dtype=torch.bfloat16
)

print("Loading LoRA adapter...")
ft_model = PeftModel.from_pretrained(base_model, ADAPTER_DIR)

tokenizer = AutoTokenizer.from_pretrained("google/gemma-3-4b-it")


In [ ]:
ft_model.print_trainable_parameters()


In [ ]:
print("\n🔎 Single WikiSQL example AFTER training:")
ex, tbl = next(iter_split("dev"))
conn = build_sqlite_from_table(tbl)
schema = make_schema_text(tbl)

gold_sql = logical_to_sql(ex["sql"], tbl)
pred_sql = generate_sql_from_llm(ex["question"], schema, ft_model)

gold_res, ge = execute_sql(conn, gold_sql)
pred_res, pe = execute_sql(conn, pred_sql)

print("Q:", ex["question"])
print("Schema:\n", schema)
print("\nGold SQL:", gold_sql)
print("Pred  SQL:", pred_sql)

print("\nGold result:", gold_res[:5])
print("Pred result:", pred_res[:5] if pe is None else f"ERROR: {pe}")


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

print("Visualization libraries imported.")

### Save Training Log


In [ ]:
log_history = trainer.state.log_history

### View Training Log

In [ ]:
import pandas as pd

# Access the training history
log_history = trainer.state.log_history

# Filter entries to include only those with a 'loss' key (indicating training steps)
training_logs = [entry for entry in log_history if 'loss' in entry]

# Extract 'loss' and 'step' values
losses = [entry['loss'] for entry in training_logs]
steps = [entry['step'] for entry in training_logs]

# Create a DataFrame for easier manipulation and plotting
training_metrics_df = pd.DataFrame({
    'step': steps,
    'loss': losses
})

print("Training metrics extracted and stored in 'training_metrics_df'.")
print(training_metrics_df.head())


### Training Visualization



In [ ]:
print(log_history)

In [ ]:
training_metrics_df['loss_ema'] = (
    training_metrics_df['loss'].ewm(span=50).mean()
)

plt.figure(figsize=(10, 6))
sns.lineplot(x='step', y='loss', data=training_metrics_df, alpha=0.4, label="Raw loss")
sns.lineplot(x='step', y='loss_ema', data=training_metrics_df, label="Smoothed EMA loss")
plt.title("Training Loss (Raw vs EMA Smoothed)")
plt.xlabel("Training Step")
plt.ylabel("Loss")
plt.grid(True)
plt.legend()
plt.show()


In [ ]:
grad_logs = [e for e in trainer.state.log_history if 'grad_norm' in e]

if grad_logs:
    grad_df = pd.DataFrame({
        'step': [e['step'] for e in grad_logs],
        'grad_norm': [e['grad_norm'] for e in grad_logs],
    })

    plt.figure(figsize=(10,6))
    sns.lineplot(data=grad_df, x='step', y='grad_norm')
    plt.title("Gradient Norm Over Training")
    plt.xlabel("Step")
    plt.ylabel("Grad Norm")
    plt.grid(True)
    plt.show()
else:
    print("No grad_norm entries logged.")


In [ ]:
plt.figure(figsize=(8,6))
sns.scatterplot(
    data=df,
    x='loss',
    y='mean_token_accuracy',
    hue='step',
    palette='viridis',
    s=120
)

plt.title("Training Phase Diagram: Loss vs Accuracy")
plt.xlabel("Loss")
plt.ylabel("Mean Token Accuracy")
plt.grid(True)
plt.show()


In [ ]:
import torch
import torch.nn.functional as F
import numpy as np

def top_tokens_from_logits(logits, tokenizer, k=10):
    probs = F.softmax(torch.tensor(logits), dim=-1)
    topk = torch.topk(probs, k)

    tokens = []
    for idx in topk.indices.tolist():
        try:
            tok = tokenizer.decode(idx)   # Gemma-style
        except Exception:
            tok = tokenizer.decode([idx]) # fallback
        tokens.append(tok)

    values = topk.values.tolist()
    return list(zip(tokens, values))

def inspect_generation(model, tokenizer, prompt, max_new_tokens=50):
    model.eval()

    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    input_ids = inputs["input_ids"]

    generated = input_ids.clone()
    logits_history = []

    for _ in range(max_new_tokens):
        with torch.no_grad():
            outputs = model(input_ids=generated)
            next_token_logits = outputs.logits[:, -1, :]  # last token logits
            logits_history.append(next_token_logits.float().cpu().numpy())

            next_token = torch.argmax(next_token_logits, dim=-1)
            generated = torch.cat([generated, next_token.unsqueeze(0)], dim=1)

            if next_token.item() == tokenizer.eos_token_id:
                break

    decoded = tokenizer.decode(generated[0], skip_special_tokens=True)
    return decoded, np.array(logits_history)


def compute_entropy_over_time(logits_history):
    entropies = []
    for logits in logits_history:
        probs = F.softmax(torch.tensor(logits), dim=-1)
        entropy = -torch.sum(probs * torch.log(probs + 1e-9)).item()
        entropies.append(entropy)
    return entropies

def top_tokens_df(logits, tokenizer, k=15):
    probs = F.softmax(torch.tensor(logits), dim=-1)
    topk = torch.topk(probs, k)
    df = pd.DataFrame({
        "token_id": topk.indices.tolist(),
        "token": [tokenizer.decode(id) for id in topk.indices.tolist()],
        "prob": topk.values.tolist()
    })
    return df
import matplotlib.pyplot as plt

prompt = processed_dataset[0]['prompt']
decoded, logits_hist = inspect_generation(ft_model, tokenizer, prompt)

entropies = compute_entropy_over_time(logits_hist)

plt.figure(figsize=(10,5))
plt.plot(entropies)
plt.title("Token Entropy Over Generation Steps")
plt.xlabel("Generation Step")
plt.ylabel("Entropy")
plt.grid(True)
plt.show()

print("\nTop tokens at first step:")
print(top_tokens_from_logits(logits_hist[0], tokenizer))
print(top_tokens_from_logits(logits_hist[1], tokenizer))
print(top_tokens_from_logits(logits_hist[2], tokenizer))

print("\nTop tokens at last step:")
print(top_tokens_from_logits(logits_hist[-1], tokenizer))


### Run Post-Training Evaluation


In [ ]:
print("\n🚀 Running post-training evaluation...")
post_training_evaluation = evaluate_execution_detailed(ft_model, n=10)

print("Post-training execution accuracy:", post_training_evaluation["exec_accuracy"])
# print_error_examples(post_training_evaluation["results"], k=3)
# print_similarity_report(post_training_evaluation["results"], k=5)

In [ ]:
import matplotlib.pyplot as plt
levs = [r["levenshtein"] for r in baseline["results"]]
plt.hist(levs, bins=20)
plt.title("Levenshtein Similarity Distribution (Baseline)")
plt.xlabel("Levenshtein Similarity")
plt.ylabel("Frequency")
plt.show()


In [ ]:
from collections import Counter

errors = Counter([
    "sql_error" if r["pred_error"] else
    "semantic_mismatch" if not r["exec_match"] else
    "correct"
    for r in baseline["results"]
])

plt.bar(errors.keys(), errors.values())
plt.title("Error Type Breakdown")
plt.show()
